<a href="https://colab.research.google.com/github/Charl35X/CSC411-Mini-Project/blob/consumer-producer/Producer_Consumer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import threading
import random
import xml.etree.ElementTree as ET
# ITstudent class to store student information
class ITstudent:
    def __init__(self, name, student_id, program, courses, marks):
        self.name = name
        self.student_id = student_id
        self.program = program
        self.courses = courses
        self.marks = marks


# Buffer class to handle shared buffer
class Buffer:
    def __init__(self):
        self.buffer = []
        self.max_size = 10
        self.empty = threading.Semaphore(0)
        self.full = threading.Semaphore(self.max_size)
        self.mutex = threading.Semaphore(1)

    def insert(self, item):
        self.full.acquire()
        self.mutex.acquire()
        self.buffer.append(item)
        self.mutex.release()
        self.empty.release()

    def remove(self):
        self.empty.acquire()
        self.mutex.acquire()
        item = self.buffer.pop(0)
        self.mutex.release()
        self.full.release()
        return item

# Producer function to generate student information and XML files
def producer(buffer):
    for i in range(1, 11):
        name = "Student " + str(i)
        student_id = str(random.randint(10000000, 99999999))
        program = "Program " + str(i)
        courses = ["Course A", "Course B", "Course C"]
        marks = [random.randint(0, 100) for _ in range(len(courses))]
        student = ITstudent(name, student_id, program, courses, marks)

        # Generate XML file
        root = ET.Element("Student")
        ET.SubElement(root, "Name").text = student.name
        ET.SubElement(root, "StudentID").text = student.student_id
        ET.SubElement(root, "Program").text = student.program
        courses_elem = ET.SubElement(root, "Courses")
        for course, mark in zip(student.courses, student.marks):
            course_elem = ET.SubElement(courses_elem, "Course")
            ET.SubElement(course_elem, "Name").text = course
            ET.SubElement(course_elem, "Mark").text = str(mark)

        tree = ET.ElementTree(root)
        tree.write("student{}.xml".format(i))

        # Insert file number into buffer
        buffer.insert(i)

# Consumer function to read XML files, calculate average marks, and print student information
def consumer(buffer):
    while True:
        file_number = buffer.remove()
        tree = ET.parse("student{}.xml".format(file_number))
        root = tree.getroot()

        name = root.find("Name").text
        student_id = root.find("StudentID").text
        program = root.find("Program").text
        courses = []
        marks = []
        for course_elem in root.find("Courses"):
            course = course_elem.find("Name").text
            mark = int(course_elem.find("Mark").text)
            courses.append(course)
            marks.append(mark)

        student = ITstudent(name, student_id, program, courses, marks)

        # Calculate average mark
        average_mark = sum(student.marks) / len(student.marks)

        # Determine pass/fail
        pass_fail = "Pass" if average_mark >= 50 else "Fail"

        # Print student information
        print("Student Name:", student.name)
        print("Student ID:", student.student_id)
        print("Program:", student.program)
        print("Courses and Marks:")
        for course, mark in zip(student.courses, student.marks):
            print(course, "-", mark)
        print("Average Mark:", average_mark)
        print("Pass/Fail:", pass_fail)
        print()

# Create buffer and start producer and consumer threads
buffer = Buffer()
producer_thread = threading.Thread(target=producer, args=(buffer,))
consumer_thread = threading.Thread(target=consumer, args=(buffer,))
producer_thread.start()
consumer_thread.start()

Student Name: Student 1
Student ID: 14370599
Program: Program 1
Courses and Marks:
Course A - 38
Course B - 47
Course C - 57
Average Mark: 47.333333333333336
Pass/Fail: Fail

Student Name: Student 2
Student ID: 60108219
Program: Program 2
Courses and Marks:
Course A - 12
Course B - 19
Course C - 4
Average Mark: 11.666666666666666
Pass/Fail: Fail

Student Name: Student 3
Student ID: 72051393
Program: Program 3
Courses and Marks:
Course A - 17
Course B - 53
Course C - 76
Average Mark: 48.666666666666664
Pass/Fail: Fail

Student Name: Student 4
Student ID: 98720600
Program: Program 4
Courses and Marks:
Course A - 9
Course B - 100
Course C - 59
Average Mark: 56.0
Pass/Fail: Pass

Student Name: Student 5
Student ID: 78504449
Program: Program 5
Courses and Marks:
Course A - 63
Course B - 10
Course C - 28
Average Mark: 33.666666666666664
Pass/Fail: Fail

Student Name: Student 6
Student ID: 72924501
Program: Program 6
Courses and Marks:
Course A - 59
Course B - 22
Course C - 70
Average Mark: 5

The `producer` function first generates a random student ID and program. Then, it generates a list of courses and marks for the student. Next, it creates an XML file that contains the student information. Finally, it inserts the file number into the buffer.

The `Buffer` class is used to store the file numbers of the XML files. The `empty` semaphore is used to indicate when the buffer is empty. The `full` semaphore is used to indicate when the buffer is full. The `mutex` semaphore is used to protect the `buffer` attribute.

The `consumer` function first removes a file number from the buffer. Then, it reads the XML file and parses it to get the student information. Next, it calculates the average mark for the student. Finally, it prints the student information.

The `threading` module is used to create and manage threads. The `random` module is used to generate random numbers. The `xml.etree.ElementTree` module is used to parse XML files.

Here is a more detailed explanation of why each component was used:

* The `ITstudent` class was used to store student information.
* The `Buffer` class was used to store the file numbers of the XML files.
* The `producer` function was used to generate student information and XML files, and then insert the file numbers into the buffer.
* The `consumer` function was used to read the XML files, calculate the average marks, and print student information.
* The `threading` module was used to create and manage threads.
* The `random` module was used to generate random numbers.
* The `xml.etree.ElementTree` module was used to parse XML files.

Here are some specific details about how the code meets the criteria:

* Wrapping and Unwrapping XML data: The code uses the xml.etree.ElementTree module to create and parse XML files. The ITstudent class stores student information in a way that is easy to serialize to XML. For example, the name attribute of the ITstudent class is stored as the Name element in the XML file.

* Implementation of concurrency, synchronization, and other aspects: The code uses the threading module to create and manage threads. The Buffer class uses semaphores to ensure that the producer and consumer threads do not interfere with each other. For example, the full semaphore is used to indicate when the buffer is full, and the empty semaphore is used to indicate when the buffer is empty.

* Run of the producer/consumer problem: The code runs the producer/consumer problem. The producer thread generates student information and XML files, and the consumer thread reads the XML files and prints student information. The producer and consumer threads run concurrently, and they use the Buffer class to share data.
